In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf
tf.random.set_seed(
    42
)


import seaborn as sns

tf_chkpt_path = 'tf_AlexNet_checkpoints/'

In [3]:
from src.helperFunctions import display_model_trainTestGraphs

# Training Set

In [4]:
# Normalizing data for 8 bit
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'images/train/',
    target_size = (227,227),
    batch_size=128,
    color_mode='rgb',
    class_mode='categorical'
)

Found 48979 images belonging to 8 classes.


# Validation Set

In [5]:
# Normalizing data for 8 bit
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    'images/val/',
    target_size = (227,227),
    batch_size=128,
    color_mode='rgb',
    class_mode='categorical'
)

Found 14175 images belonging to 8 classes.


# Test Set

In [6]:
# Normalizing data for 8 bit
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'images/test/',
    target_size = (227,227),
    batch_size=128,
    color_mode='rgb',
    class_mode='categorical'
)

Found 1793 images belonging to 8 classes.


# Modeling

In [7]:
early_stop = EarlyStopping(monitor='val_loss', 
                           min_delta=1e-8,
                           verbose=1,
                           patience = 5,
                           restore_best_weights=True,
                           mode='min')

# Model - AlexNet

In [8]:
model = models.Sequential()

model.add(layers.Conv2D(filters=96, kernel_size=11, strides=(4, 4), padding='valid', activation='relu',
                        input_shape=(227, 227, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2),padding='valid'))

model.add(layers.Conv2D(filters=256, kernel_size=(
        5, 5), strides=(1, 1), activation='relu', padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))

model.add(layers.Conv2D(filters=384, kernel_size=(
        3, 3), strides=(1, 1), activation='relu', padding="same"))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(filters=384, kernel_size=(
        3, 3), strides=(1, 1), activation='relu', padding="same"))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(filters=256, kernel_size=(
        3, 3), strides=(1, 1), activation='relu', padding="same"))
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))


model.add(layers.Flatten())
model.add(layers.Dense(4096, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation = 'relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(8, activation='softmax'))


model.compile(optimizer = optimizers.Adam(learning_rate=0.0001),
              loss= losses.CategoricalCrossentropy(),
              metrics = ['accuracy', Precision(), Recall()])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 27, 27, 256)      1024      
 hNormalization)                                                 
                                                        

In [9]:
results = model.fit(train_generator,
                    steps_per_epoch=200,  # Number of training samples to pull per epoch
                    epochs=150,  # Number of epochs
                    validation_data=val_generator,
                    validation_freq=1,
                    validation_steps=100,  # Number of validation samples to run validation on
#                     verbose=1,
#                     callbacks = [early_stop]
                    )

Epoch 1/150
200/200 [==============================] - 37s 161ms/step - loss: 0.7654 - accuracy: 0.8212 - precision: 0.8437 - recall: 0.7952 - val_loss: 0.9975 - val_accuracy: 0.7902 - val_precision: 0.7902 - val_recall: 0.7902
Epoch 2/150
200/200 [==============================] - 32s 162ms/step - loss: 0.4946 - accuracy: 0.8531 - precision: 0.8741 - recall: 0.8301 - val_loss: 1.0615 - val_accuracy: 0.7913 - val_precision: 0.7913 - val_recall: 0.7912
Epoch 3/150
200/200 [==============================] - 32s 158ms/step - loss: 0.4155 - accuracy: 0.8744 - precision: 0.8903 - recall: 0.8557 - val_loss: 3.1947 - val_accuracy: 0.2430 - val_precision: 0.2302 - val_recall: 0.2185
Epoch 4/150
200/200 [==============================] - 31s 157ms/step - loss: 0.3611 - accuracy: 0.8899 - precision: 0.9037 - recall: 0.8749 - val_loss: 1.1231 - val_accuracy: 0.5763 - val_precision: 0.6306 - val_recall: 0.4969
Epoch 5/150
200/200 [==============================] - 32s 159ms/step - loss: 0.3426 - a

KeyboardInterrupt: 

Interrupted because it val_loss was not converging